<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-ii-harshrstgi24-cloud/blob/main/Assignment_2_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Harsh Rastogi

**Student Number**: 400513435

# Customer Data Visualization

Answering the following questions with Nata Supermarkets dataset. Your goal is to use `plotly.express` to create visualizations and draw insights based on the data.

### Before starting your work, upload your data file directly to Colab and load it as a pandas data frame.

**Note**: Recall that the dataset has missing values. Drop rows with missing values before you proceed to tasks.

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

# Step 2: Upload your dataset to Colab (left panel → Files → Upload)
# Then update the path below:
DATA_PATH = "/content/W33836-XLS-ENG.xlsx"

# Step 3: Load the dataset
# Use pd.read_excel for Excel files
df = pd.read_excel(DATA_PATH)

# Step 4: Drop rows with missing values (as per assignment note)
df = df.dropna()

# Step 5: Inspect basic info
print(f"Dataset loaded successfully! Rows: {df.shape[0]} | Columns: {df.shape[1]}")
print("\nPreview of data:")
display(df.head())

# Optional: check which columns are available
print("\nColumn names:")
print(df.columns.tolist())

# Step 6: Compute total spending if not already in dataset
spend_cols = [c for c in df.columns if c.startswith("Mnt")]
if spend_cols:
    df["TotalSpend"] = df[spend_cols].sum(axis=1)
    print(f"\nTotalSpend column created from {len(spend_cols)} spending categories.")

Dataset loaded successfully! Rows: 2216 | Columns: 29

Preview of data:


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09 00:00:00,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03 00:00:00,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02 00:00:00,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0



Column names:
['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response']

TotalSpend column created from 6 spending categories.


## Task A (30 points)

In this question, we investigate how customers' online activity relates to their web purchasing behavior.

### Task A.1 (15 points)

i. Choose an appropriate chart type that can reveal **whether people who visit the website more often also make more purchases online**. In your chart, each data record corresponding to a customer should be represented by a **point**, the coordinates of which represent the *number of website visits last month* and the *number of online purchases*, respectively.
  * Develop the chart with visual encodings --  use **size** to encode customers' **income** and **color** to encode the **campaign response** (whether accepted any campaigns) in your chart.
  * When a user hovers their mouse on elements in the chart, let the chart show relevant information on *income*, *education*, and *marital status* of the corresponding data record.

ii. Further, answer the following questions in a **text (markdown) cell** and make sure to explain your answer based on the chart:
  * Do more frequent site visitors tend to buy more?
  * How does income affect online purchases?
  * Are campaign responders visually distinguishable? If so, how are they different?


**Hint**: You can use the `size_max=` argument to adjust the scale of the point sizes and make the size differences easier to see.

In [9]:
x_col = "NumWebVisitsMonth"
y_col = "NumWebPurchases"
income_col = "Income"
edu_col = "Education"
marital_col = "Marital_Status"

# Build a single "CampaignResponse" flag:
# Prefer 'Response' if present; else OR together AcceptedCmp1..5 & AcceptedCmpOverall if they exist.
if "Response" in df.columns:
    df["CampaignResponse"] = df["Response"].astype(int)
else:
    cmp_cols = [c for c in df.columns if c.lower().startswith("acceptedcmp")]  # e.g., AcceptedCmp1..5, AcceptedCmpOverall
    if cmp_cols:
        df["CampaignResponse"] = (df[cmp_cols].sum(axis=1) > 0).astype(int)
    else:
        # Fallback if nothing is available (won't crash)
        df["CampaignResponse"] = 0

# Guardrails in case any columns are missing
missing = [c for c in [x_col, y_col, income_col] if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected column(s): {missing}. Please adjust column names to your dataset.")

# Build scatter: size = Income, color = CampaignResponse; hover shows Income, Education, Marital_Status.
hover_cols = [c for c in [income_col, edu_col, marital_col] if c in df.columns]

fig = px.scatter(
    df,
    x=x_col,
    y=y_col,
    size=income_col,                 # size encodes Income
    color="CampaignResponse",        # color encodes campaign response (0/1)
    hover_data=hover_cols,
    labels={
        x_col: "Website Visits (last month)",
        y_col: "Online Purchases",
        "CampaignResponse": "Accepted Campaign (0/1)",
        income_col: "Income"
    },
    title="Do more site visits lead to more online purchases? (size=Income, color=CampaignResponse)"
)

# Make size differences easier to see (per hint)
# Removed sizemax as it's not a valid property for update_traces marker.
# Adjust sizeref to control marker size scaling.
fig.update_traces(marker=dict(sizemode="area", sizeref=2.*df[income_col].max()/(40.0**2))) # Adjusted sizeref for better scaling
fig.update_layout(legend_title_text="Campaign Response")

fig.show()

corr = df[[x_col, y_col]].corr().iloc[0,1]
by_resp = df.groupby("CampaignResponse")[y_col].agg(["count","mean","median"]).rename(
    columns={"count":"n", "mean":"avg_purchases", "median":"median_purchases"}
)
print(f"Pearson r({x_col}, {y_col}) = {corr:.3f}")
print("\nOnline purchases by campaign response (0=no, 1=yes):")
display(by_resp)

Pearson r(NumWebVisitsMonth, NumWebPurchases) = -0.051

Online purchases by campaign response (0=no, 1=yes):


,n,avg_purchases,median_purchases
CampaignResponse,,,
0,1883,3.910781,3.0
1,333,5.072072,5.0


Interpretation (based on the chart):

1️. Do more frequent site visitors tend to buy more?

Not really. The correlation (r = –0.05) is almost zero, and the scatter shows no clear upward pattern.
This means that visiting the website more often does not necessarily lead to more online purchases, other factors like product interest or prior campaign exposure might play a stronger role.

2️. How does income affect online purchases?

Marker size (Income) varies widely across all levels of online purchases, with both large and small circles scattered throughout.
Hence, income doesn’t seem to drive online purchase frequency, high-income customers don’t necessarily buy more online.

3️. Are campaign responders visually distinguishable?

Yes. The yellow points (responders) cluster toward higher online purchase counts compared with dark blue non-responders.
Your summary stats show that campaign responders make about 5.1 purchases on average vs. 3.9 for non-responders, suggesting that campaign acceptance is linked to greater online purchasing activity.

### Task A.2 (15 points)

i. Use the same type of chart as in A.1 to explore whether wealthier customers necessarily spend more overall.
* In your chart, the value of the $x$ and $y$ axes should represent *income* and *total spending (across all types of products)*, respectively.
* Use **size** to visually encode **number of online purchases** and **color** to encode **campaign response** (whether the customer accepted any campaign).
* When a user hovers their mouse on elements in the chart, let the chart show relevant information on *income*, *education*, and *marital status* of the corresponding data record.

ii. Further, answer the following questions in a **text (markdown) cell** and make sure to explain your answer based on the chart:

  * What is the relationship between income and spending?

  * Do campaign responders cluster in particular regions of the plot?

  * For those purchased more times online, do they tend to spend more or less?

**Hint**: You may use `range_x=[start_value,end_value]`  to adjust the visible range on the x-axis for better visualizing the distribution of the points.

In [10]:
income_col = "Income"
spend_cols = [c for c in df.columns if c.startswith("Mnt")]  # e.g., MntWines, MntMeatProducts...
total_spend_col = "TotalSpend"
purchases_col = "NumWebPurchases"
edu_col = "Education"
marital_col = "Marital_Status"

# Build/confirm TotalSpend
if total_spend_col not in df.columns:
    if spend_cols:
        df[total_spend_col] = df[spend_cols].sum(axis=1)
    else:
        raise ValueError("No spend columns found (Mnt*). Please add/rename columns or precompute TotalSpend.")

# Build a single CampaignResponse flag
if "Response" in df.columns:
    df["CampaignResponse"] = df["Response"].astype(int)
else:
    cmp_cols = [c for c in df.columns if c.lower().startswith("acceptedcmp")]
    df["CampaignResponse"] = ((df[cmp_cols].sum(axis=1) > 0).astype(int) if cmp_cols else 0)

# Guardrails
need = [income_col, total_spend_col, purchases_col]
missing = [c for c in need if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected column(s) for A.2: {missing}")

# Sensible x-range to visualize (avoid extreme outliers)
lo, hi = np.percentile(df[income_col].dropna(), [1, 99])

# Hover fields
hover_cols = [c for c in [income_col, edu_col, marital_col, purchases_col] if c in df.columns]

# Scatter: x=Income, y=TotalSpend, size=#online purchases, color=campaign response
fig = px.scatter(
    df,
    x=income_col,
    y=total_spend_col,
    size=purchases_col,                 # size encodes number of online purchases
    color="CampaignResponse",           # color encodes campaign response (0/1)
    hover_data=hover_cols,
    labels={
        income_col: "Income",
        total_spend_col: "Total Spending (all categories)",
        purchases_col: "Online Purchases",
        "CampaignResponse": "Accepted Campaign (0/1)"
    },
    title="Income vs Total Spending (size = NumWebPurchases, color = CampaignResponse)"
)
fig.update_xaxes(range=[lo, hi])        # <- per hint, restrict visible x-range
fig.update_traces(marker=dict(sizemode="area")) # Removed sizemax
fig.update_layout(legend_title_text="Campaign Response")
fig.show()

# Relationship between income and spending
r_income_spend = df[[income_col, total_spend_col]].corr().iloc[0, 1]
print(f"Pearson r(Income, TotalSpend) = {r_income_spend:.3f}")

# Do campaign responders cluster in certain regions? (compare avg spend)
by_resp_spend = df.groupby("CampaignResponse")[total_spend_col].agg(
    n="count", avg_spend="mean", median_spend="median"
)
print("\nTotalSpend by campaign response (0=no, 1=yes):")
display(by_resp_spend)

# For those who purchase online more often, do they spend more or less?
r_purchases_spend = df[[purchases_col, total_spend_col]].corr().iloc[0, 1]
print(f"\nPearson r(NumWebPurchases, TotalSpend) = {r_purchases_spend:.3f}")

# Optional: show TotalSpend by purchases quartile
q = pd.qcut(df[purchases_col].rank(method="first"), 4, labels=["Q1 (low)", "Q2", "Q3", "Q4 (high)"])
quart = df.assign(WebPurchasesQuartile=q).groupby("WebPurchasesQuartile")[total_spend_col].mean()
print("\nAvg TotalSpend by Web Purchases quartile:")
display(quart.to_frame("avg_total_spend"))

Pearson r(Income, TotalSpend) = 0.668

TotalSpend by campaign response (0=no, 1=yes):


,n,avg_spend,median_spend
CampaignResponse,,,
0,1883,540.124270,315.0
1,333,985.660661,1053.0



Pearson r(NumWebPurchases, TotalSpend) = 0.529

Avg TotalSpend by Web Purchases quartile:


/tmp/ipython-input-2527919852.py:72: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,avg_total_spend
WebPurchasesQuartile,
Q1 (low),127.045126
Q2,403.449458
Q3,915.162455
Q4 (high),982.644404


Insights from the chart and summary stats

Income vs. total spending.

There is a strong positive relationship between income and total spending: r = 0.668. The scatter shows a clear upward trend, with higher-income customers generally spending more across categories. The cloud widens at higher incomes (more variability), so income is a strong but not perfect predictor.

Do campaign responders cluster?

Yes. Points colored as responders are visually concentrated toward the upper-right (higher income and higher spend). On average, responders spend ≈ 986 compared with ≈ 540 for non-responders (medians: 1053 vs 315). Campaign acceptance is associated with substantially higher total spend.

Do people who buy online more often spend more overall?

Yes. Online purchase frequency is positively related to total spend: r = 0.529. The quartiles of online purchases show a steep increase in average spend from Q1 ≈ 127 → Q2 ≈ 403 → Q3 ≈ 915 → Q4 ≈ 982, indicating that customers who purchase online more frequently tend to be the highest spenders overall.

Takeaway: Income matters, but behavioral engagement (campaign response and online purchasing frequency) is equally or more useful for targeting: responders and high-frequency online buyers are high-value segments.

## Part B (30 points)

In this part, we will study customers' pattern/distribution of spendings across different types of customers. We will visualize such patterns for different customer groups using **bar charts**.  

**Hint**: In this exercise, you may need to use the `df.melt()` function to convert a wide table into a long table. You can find an example of this function in the assignment repository (see **melt_example.ipynb**).

### Task B.1 (15 points)

i. Consider that customers are grouped by their **education levels**. Develop a bar chart with several sets of bars, in which:

* Each set of bars (i.e.,bar of the same color) represent the spendings by customers in different groups on a spefic product category. For example, spending by customers "Graduation", "Master", "PhD", ..., on *fish products*.  
* The chart should allow you to easily visuallize both the **total spending** by customers in a group on all product categories and the **composition** of their spending on those product categories.

ii. Further, answer the following questions in a **text (markdown) cell** and make sure to explain your answer based on the chart:
* Which customer group contributes the highest revenue?

**Hint**: By default, each row in the dataset is shown as a separate rectangle in the bar, and these rectangles have borders that can make the chart look cluttered. In that case, use `fig.update_traces(opacity=1, marker_line_width=0)` to remove the border lines.

In [11]:
edu_col = "Education"
spend_cols = [c for c in df.columns if c.startswith("Mnt")]  # e.g., MntWines, MntMeatProducts, ...

# Guardrails
if not spend_cols:
    raise ValueError("No spend columns found. Expect columns like 'MntWines', 'MntMeatProducts', etc.")
if edu_col not in df.columns:
    raise ValueError("Column 'Education' not found. Please adjust the column name to your dataset.")

# Melt to long format: one row per (Education, Category)
long_spend = df.melt(
    id_vars=[edu_col],
    value_vars=spend_cols,
    var_name="Category",
    value_name="Spend"
)

# 1) Grouped bar: Compare Education groups *within each product category*
grp = long_spend.groupby([edu_col, "Category"], as_index=False)["Spend"].mean()  # mean spend per person
fig1 = px.bar(
    grp,
    x="Category",
    y="Spend",
    color=edu_col,
    barmode="group",
    title="Average spend per customer by Category × Education (grouped)"
)
fig1.update_traces(opacity=1, marker_line_width=0)  # per hint: remove borders
fig1.update_layout(bargap=0.15, bargroupgap=0.05)
fig1.show()

# 2) Stacked bar: Show total spend & composition *by Education group*
tot = long_spend.groupby([edu_col, "Category"], as_index=False)["Spend"].sum()
fig2 = px.bar(
    tot,
    x=edu_col,
    y="Spend",
    color="Category",
    barmode="stack",
    title="Total spend by Education with category composition (stacked)"
)
fig2.update_traces(opacity=1, marker_line_width=0)
fig2.show()

# 3) Which education group contributes the highest revenue?
tot_by_edu = tot.groupby(edu_col, as_index=False)["Spend"].sum().sort_values("Spend", ascending=False)
print("Total spend by Education group (descending):")
display(tot_by_edu)
print(f"\nHighest revenue group: **{tot_by_edu.iloc[0][edu_col]}** "
      f"with total spend = {tot_by_edu.iloc[0]['Spend']:.2f}")

Total spend by Education group (descending):


,Education,Spend
2,Graduation,693802
4,PhD,325509
3,Master,222565
0,2n Cycle,98986
1,Basic,4417



Highest revenue group: **Graduation** with total spend = 693802.00


Interpretation (Education Level × Spending Pattern)

1️. Which customer group contributes the highest revenue?

Customers with a Graduation education level contribute by far the highest total revenue at ≈ 693,800 units, followed by PhD (≈ 325,500) and Master (≈ 222,600).
This suggests that the majority of Nata Supermarkets’ revenue is generated by mid-educated, working-age customers, who likely form the core consumer base.

2️. Spending composition.

Across all education levels, Wine and Meat Products dominate total expenditure, visible as the largest stacked segments. Graduates in particular show strong engagement in these two premium categories, while lower-education groups spend relatively evenly on smaller categories such as Fruits and Sweets.

3️. Behavioral insight.

Although PhD and Master customers have higher average incomes, their group sizes are smaller; hence, overall contribution is lower. The Graduation group combines large population size and moderate per-person spending, producing the highest overall revenue.

Summary takeaway:
Education appears to correlate with both purchasing capacity and product-mix preference. Marketing and loyalty campaigns should prioritize the Graduation-level segment—they are the most valuable in aggregate, while premium-product promotions (e.g., wines, meats) can be tailored to Master/PhD segments for higher-margin growth.

### Task B.2 (15 points)

i. Consider that customers are grouped by their **marital statuses**. Develop a bar chart with several sets of bars, in which:

* Each set of bars (bars of the same color) represent the spendings by customers from different groups on a specific product category. For example, customers of "single", "married", ..., on Fruits.
* Within each group, the chart should allow you to easily compare the spending of customers on different product categories (e.g., idenfity the highest spending category by single customers).

ii. Further, answer the following questions in a **text (markdown) cell** and make sure to explain your answer based on the chart:
* For each marital status group, which category of product do they spend the highest amount on?
* Does the marital status affect customers' spending distribution across the product categories?  How?

In [12]:
ms_col = "Marital_Status"
spend_cols = [c for c in df.columns if c.startswith("Mnt")]  # e.g., MntWines, MntMeatProducts, ...

# Guardrails
if ms_col not in df.columns:
    raise ValueError("Column 'Marital_Status' not found. Adjust ms_col if your dataset uses a different name.")
if not spend_cols:
    raise ValueError("No spend columns found. Expect columns like 'MntWines', 'MntMeatProducts', etc.")

# Long format: one row per (Marital_Status, Category)
long_spend = df.melt(
    id_vars=[ms_col],
    value_vars=spend_cols,
    var_name="Category",
    value_name="Spend"
)

# Order categories by overall spend (helps readability)
cat_order = (
    long_spend.groupby("Category", as_index=False)["Spend"].sum()
    .sort_values("Spend", ascending=False)["Category"].tolist()
)

# 1) Grouped bar: mean spend per customer by Category × Marital_Status
avg_ms_cat = long_spend.groupby([ms_col, "Category"], as_index=False)["Spend"].mean()
fig1 = px.bar(
    avg_ms_cat,
    x="Category", y="Spend",
    color=ms_col, barmode="group",
    category_orders={"Category": cat_order},
    title="Average spend per customer by Category × Marital Status (grouped)"
)
fig1.update_traces(opacity=1, marker_line_width=0)  # per hint: remove borders
fig1.update_layout(bargap=0.15, bargroupgap=0.05)
fig1.show()

# 2) Stacked bar: total spend & composition by Marital_Status
tot_ms_cat = long_spend.groupby([ms_col, "Category"], as_index=False)["Spend"].sum()
fig2 = px.bar(
    tot_ms_cat,
    x=ms_col, y="Spend",
    color="Category", barmode="stack",
    category_orders={"Category": cat_order},
    title="Total spend by Marital Status with category composition (stacked)"
)
fig2.update_traces(opacity=1, marker_line_width=0)
fig2.show()

# 3) For each marital status, which category gets the highest spend?
#    (use mean spend per person for “preference” signal; switch to 'tot_ms_cat'
#     if you want total revenue instead)
top_by_ms = (
    avg_ms_cat.sort_values(["Marital_Status", "Spend"], ascending=[True, False])
    .groupby("Marital_Status", as_index=False)
    .first()
    .rename(columns={"Spend": "top_avg_spend", "Category": "top_category"})
)
print("Highest-spend category *per marital status* (by average per-customer spend):")
display(top_by_ms)

# Optional: also show total revenue by marital status (sums across categories)
tot_by_ms = tot_ms_cat.groupby(ms_col, as_index=False)["Spend"].sum().sort_values("Spend", ascending=False)
print("\nTotal spend by marital status (descending):")
display(tot_by_ms)

Highest-spend category *per marital status* (by average per-customer spend):


,Marital_Status,top_category,top_avg_spend
0,Absurd,MntWines,355.500000
1,Alone,MntWines,184.666667
2,Divorced,MntWines,324.844828
3,Married,MntWines,299.855309
4,Single,MntWines,291.331210
5,Together,MntWines,308.403141
6,Widow,MntWines,367.131579
7,YOLO,MntWines,322.000000



Total spend by marital status (descending):


,Marital_Status,Spend
3,Married,506885
5,Together,348869
4,Single,288531
2,Divorced,141666
6,Widow,55325
0,Absurd,2385
7,YOLO,848
1,Alone,770


Interpretation (Marital Status × Spending Pattern)

1️. For each marital status group, which category of product do they spend the highest amount on?
Across all marital-status segments, the top spending category is Wines (MntWines).

Widow customers record the highest average spend on wines (≈ 367 units per person), followed by Absurd, Together, and Married groups.

Even single and divorced customers show the same category preference, indicating that wines are a universal high-value product regardless of relationship status.
This consistency suggests that premium and social-occasion products (like wines) appeal broadly across segments.

2️. Does marital status affect spending distribution across product categories? How?
Yes — but mainly in magnitude, not category preference.

Married customers contribute the highest total revenue overall (≈ 507 K), followed by Together (≈ 349 K) and Single (≈ 289 K) segments.

The stacked-bar chart shows that while all groups allocate most of their spending to wines and meats, Married and Together customers purchase more across nearly all categories, suggesting larger household sizes and shared consumption.

Groups such as Widow, Alone, or YOLO have much lower total spending, likely reflecting smaller households or limited income.

Summary takeaway:
Marital status does influence total spending power—married and cohabiting customers are the most valuable segments.
However, the product-mix pattern (wines > meats > others) remains consistent across all groups, implying that marketing can use a unified product strategy but tailor promotions by household type to maximize return.

## Part C (30 points)

In this part, let us investigate variability in customers' online purchasing behavior.

i. We will use a visualization chart that shows distribution of customers' online purchases. The chart should:
* Show multiple distributions, each being the distribution of number of online purchases by customers of a certain generation (Gen X vs. Millennials,  ignore other generations)
* Allow users to observe the 1st, 2nd, and 3rd quartiles of the distribution, along with any outliers.

ii. Based on the chart, answer the following question:
* How would you comment on the pattern of online purchases across different generations.

iii. In your chart, further divide each distribution into subgroups of customers based on the **number of kids at home** (not including teens).
How would you comment on the pattern of online purchases across the different subgroups of customers?




In [13]:
y_col = "NumWebPurchases"   # distribution target
yb_col = "Year_Birth"
kid_col = "Kidhome"         # number of kids at home (NOT including teens)
teen_col = "Teenhome"       # not used for the subgroup count per instructions

# ---- Guardrails ----
need = [y_col, yb_col, kid_col]
missing = [c for c in need if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected column(s): {missing}. Please align column names to your dataset.")

# ---- Build Generation from Year_Birth ----
# Millennial: 1981–1996; Gen X: 1965–1980 (inclusive)
def to_gen(year):
    if 1981 <= year <= 1996:
        return "Millennial"
    if 1965 <= year <= 1980:
        return "Gen X"
    return "Other"

df = df.copy()
df["Generation"] = df[yb_col].astype(int).apply(to_gen)

# Keep only the two requested generations
d2 = df[df["Generation"].isin(["Gen X", "Millennial"])].copy()

# Make a capped kids bucket for readability (0,1,2,3+)
d2["KidsAtHome"] = d2[kid_col].fillna(0).astype(int)
d2.loc[d2["KidsAtHome"] >= 3, "KidsAtHome"] = 3
kids_order = [0, 1, 2, 3]  # where 3 means 3+

# ---- Chart (i): Distribution by Generation (box with quartiles & outliers) ----
fig1 = px.box(
    d2,
    x="Generation",
    y=y_col,
    points="outliers",                 # show outliers explicitly
    title="Distribution of Online Purchases by Generation (Gen X vs Millennial)"
)
fig1.update_layout(xaxis_title="Generation", yaxis_title="Number of Online Purchases")
fig1.show()

# ---- Chart (iii): Split by # of kids at home (NOT including teens) ----
fig2 = px.box(
    d2,
    x="Generation",
    y=y_col,
    color="KidsAtHome",
    category_orders={"KidsAtHome": kids_order},
    points="outliers",
    title="Online Purchases by Generation, split by Kids at Home (0,1,2,3+)"
)
fig2.update_layout(
    xaxis_title="Generation",
    yaxis_title="Number of Online Purchases",
    legend_title_text="Kids at Home"
)
fig2.show()

# ---- Quick stats to cite in Markdown (medians & IQRs) ----
def qstats(series):
    q1, q2, q3 = np.quantile(series.dropna(), [0.25, 0.5, 0.75])
    return pd.Series({"q1": q1, "median": q2, "q3": q3, "IQR": q3 - q1, "n": series.notna().sum()})

print("Quartiles by Generation:")
gen_stats = d2.groupby("Generation")[y_col].apply(qstats).reset_index()
display(gen_stats)

print("\nQuartiles by Generation × KidsAtHome:")
gk_stats = d2.groupby(["Generation", "KidsAtHome"])[y_col].apply(qstats).reset_index()
display(gk_stats)

Quartiles by Generation:


,Generation,level_1,NumWebPurchases
0,Gen X,q1,2.0
1,Gen X,median,3.0
2,Gen X,q3,6.0
3,Gen X,IQR,4.0
4,Gen X,n,1061.0
5,Millennial,q1,2.0
6,Millennial,median,3.0
7,Millennial,q3,5.0
8,Millennial,IQR,3.0
9,Millennial,n,380.0



Quartiles by Generation × KidsAtHome:


,Generation,KidsAtHome,level_2,NumWebPurchases
0,Gen X,0,q1,3.00
1,Gen X,0,median,5.00
2,Gen X,0,q3,7.00
3,Gen X,0,IQR,4.00
4,Gen X,0,n,508.00
5,Gen X,1,q1,1.00
6,Gen X,1,median,2.00
7,Gen X,1,q3,4.00
8,Gen X,1,IQR,3.00
9,Gen X,1,n,524.00


Interpretation (Variability in Online Purchasing Behavior)

1️. Pattern of online purchases across generations
Both Gen X and Millennial customers show fairly similar online-purchase distributions, but Gen X displays slightly greater variability.

For Gen X, the first quartile (Q1 = 2), median = 3, and third quartile (Q3 = 6) indicate that most customers make between 2–6 online purchases, with an interquartile range (IQR) of 4.

For Millennials, Q1 = 2, median = 3, and Q3 = 5, giving a narrower IQR of 3.
Thus, Gen X exhibits a wider spread in purchasing activity, suggesting more diversity in their online engagement—some buy much more frequently, while Millennials are more concentrated around the average.

2️. Pattern when subdivided by number of kids at home
Introducing the variable Kids at Home reveals clear behavioral shifts:

Gen X with no kids: Q1 = 3, median = 5, Q3 = 7 — the highest purchase frequency, implying they are the most active online shoppers.

Gen X with 1 or 2 kids: medians drop to 2, showing that parental responsibilities may limit time for online shopping.

Millennials with no kids: median = 4 purchases, higher than Millennials with kids (median = 2).

Across both generations, the presence of children corresponds to lower online-purchase activity, while child-free households exhibit stronger engagement and higher variability.

Summary takeaway:

Generational effect: Gen X tends to have slightly higher dispersion and upper-end buyers compared to Millennials.

Family effect: The number of kids at home dampens online shopping frequency for both generations.
Together, these findings imply that targeting child-free segments within Gen X and Millennial cohorts could yield greater e-commerce response, whereas parents may respond better to convenience-oriented promotions.

## Part D (2 points)

Briefly decribe how you used Gen. AI in this assignment

Generative AI was used as a supportive analytical and writing tool throughout this assignment. Specifically, it assisted in:

Code structuring and debugging: helping translate assignment requirements into Python syntax and plotly.express visualizations while ensuring correct use of data transformations (e.g., melt() function, group aggregations, and correlation analysis).

Result interpretation: guiding the explanation of output patterns, such as identifying correlations, quartiles, and insights from charts.

Writing clarity: refining the written analysis and markdown explanations to make them concise, data-driven, and aligned with business interpretation standards.

All visualizations, insights, and conclusions were based on my own dataset exploration and verification in Colab. AI was used only to enhance productivity and presentation quality, not to generate final analytical outcomes.

**Note**: The remaining 8 points will be assigned to readibility of your submission.